**Unsupervised 1**

# K-Means Clustering

## Learning objectives

- Understand K-means clustering as an unsupervised learning technique
- Learn how to prepare data for clustering analysis
- Apply the elbow method to determine optimal cluster numbers
- Interpret clusters for policy insights

### London crime profiles by area

Policy context: London's 1000+ neighbourhoods (MSOAs) experience different types and volumes of crime. While total crime rates matter, the *composition* of crime types reveals important patterns:
- Some areas might struggle with street crimes (theft, robbery)
- Others with anti-social behaviour
- Some with property crimes (burglary, vehicle crime)

**Setup: Import required libraries**

In [7]:
import pandas as pd
import altair as alt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

---

<br>
<br>

### Step 1: Load and explore the data

`ldn_crimes_msoa.csv` contains month crime totals by type for all MSOA (census boundaries for areas of between 5000-1500 people) areas in London, covering the period September 2022 to 2025. See [here](https://www.ons.gov.uk/methodology/geography/ukgeographies/statisticalgeographies) for ONS explanation.

In [ ]:
data = pd.read_csv('clean-data/ldn_crimes_msoa.csv')                    # Crime counts by MSOA
# data = pd.read_csv('clean-data/ldn_crimes_agg_202209_202509.csv')     # For crimes by LSOA

print(f"\nWe have {data['MSOA code'].nunique():,} MSOA neighborhoods")
print(f"Dataset shape: {data.shape}\n")
data.head(8)  # Print first few rows


We have 1,002 MSOA neighborhoods
Dataset shape: (395267, 5)



,MSOA code,MSOA name,Month,Crime type,count
0,E02000001,City of London 001,2022-09,Anti-social behaviour,10
1,E02000001,City of London 001,2022-09,Bicycle theft,2
2,E02000001,City of London 001,2022-09,Burglary,3
3,E02000001,City of London 001,2022-09,Criminal damage and arson,2
4,E02000001,City of London 001,2022-09,Drugs,2
5,E02000001,City of London 001,2022-09,Other theft,37
6,E02000001,City of London 001,2022-09,Public order,4
7,E02000001,City of London 001,2022-09,Robbery,6


So, data is already aggregated to the MSOA neighbourhood level, for monthly crime totals for each crime type.

<br>

Explore data dimensions and crime totals

In [9]:
print(f"Date range: {data['Month'].min()} to {data['Month'].max()} \t({data['Month'].nunique()} months)\n")

# How many crimes of each type? Group by 
print(data.groupby(['Crime type'])['count'].sum().sort_values(ascending=False))

Date range: 2022-09 to 2025-08 	(36 months)

Crime type
Violence and sexual offences    754663
Anti-social behaviour           684135
Other theft                     339323
Vehicle crime                   287011
Theft from the person           238252
Shoplifting                     206338
Public order                    164277
Criminal damage and arson       162643
Burglary                        153800
Drugs                           118413
Robbery                          93568
Bicycle theft                    44578
Other crime                      33455
Possession of weapons            14132
Name: count, dtype: int64


<br>
<br>

### Step 2: Prepare Data for Clustering

**Important:** We need to transform our time-series data and normalise (scale) the values.

**2.1** Transform data into feature matrix (i.e. neighbourhood profiles):
- Each row = one neighbourhood (MSOA)
- Each column = one crime type
- Values = how much of that crime happens there (we could choose average monthly totals, sum etc)

In [11]:
# Create a pivot table: neighbourhoods x crime types
crime_matrix = data.pivot_table(
    index=['MSOA code'],
    columns='Crime type',
    values='count',
    fill_value=0,       # Fill any missing values with 0
    aggfunc='sum'        # Sum counts over time
)
crime_matrix

Crime type,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
MSOA code,,,,,,,,,,,,,,
E02000001,186,118,137,122,92,12,1445,7,163,165,498,1086,111,551
E02000002,681,7,63,137,241,29,163,15,145,45,29,23,300,880
E02000003,741,17,170,246,113,40,265,14,208,136,123,62,410,1067
E02000004,234,4,63,71,75,14,74,5,74,19,72,23,168,442
E02000005,485,21,123,182,96,35,167,16,152,88,239,40,302,961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E02007111,1105,412,491,291,699,43,1896,67,387,618,430,2749,342,2043
E02007112,317,4,47,54,46,13,123,4,55,15,6,57,184,331
E02007113,719,41,126,146,144,28,280,11,182,96,73,137,337,726


<br>
<br>

**Visualise**
- Left chart: histogram distribution of crime totals per MSOA
- Right chart: bar chart of crime totals per type

In [12]:
# Calculate total crimes per MSOA
total_crimes_per_msoa = crime_matrix.sum(axis=1).reset_index(name='total')

panel_left = alt.Chart(total_crimes_per_msoa).mark_bar(opacity=0.8, binSpacing=0).encode(
    x=alt.X('binned_total:Q').title('Total crimes (3-year period)').scale(domain=[0, 9e4]),
    y=alt.Y('count()').title('Number of MSOAs'),
    tooltip=[alt.Tooltip('count()', title='Number of MSOAs'), alt.Tooltip('bin_range:N', title='Binned range')]
).properties(width=330, height=300, title=alt.TitleParams(text='Distribution of total crimes per MSOA', subtitle='(Sep 2022 to Aug 2025)')
).transform_bin(
    'binned_total', field='total', bin=alt.Bin(maxbins=100)
).transform_calculate(
    bin_range = "datum.binned_total + ' to ' + datum.binned_total_end + ' reported crimes'"
).interactive()

# Calculate total crimes per crime type
crime_totals = crime_matrix.sum(axis=0).reset_index(name='total')

panel_right = alt.Chart(crime_totals).mark_bar().encode(
    x=alt.X('total:Q').title('Total incidents'),
    y=alt.Y('Crime type:O').sort('-x').title(''),
    tooltip=['Crime type', 'total']
).properties(
    width=330,
    height=300,
    title=alt.TitleParams(text='Crime type frequency distribution', subtitle='(Sep 2022 to Aug 2025)')
)

alt.hconcat(panel_left, panel_right)

alt.HConcatChart(...)

<br>
<br>

#### Important decision: clustering absolute or proportional crime profiles

We have two clustering options:
1. **Absolute counts**: Cluster by raw crime numbers (finds high vs. low crime areas)
2. **Proportional profiles**: Cluster by crime composition (finds areas with similar crime patterns regardless of volume)

-> We'll use **proportional profiles** to identify areas. 


<br>

**2.2** We need to normalise the data so that:
- Areas with more total crime don't dominate
- Each crime type is weighted equally
- We're comparing crime *patterns*, not crime *volume*

In [13]:
# Convert to percentages (what % of area's crime is each type?) 
crime_matrix_percentages = crime_matrix.div(crime_matrix.sum(axis=1), axis=0)  # Each row sums to 1

# Handle areas with no crime (avoid division by zero)
crime_matrix_percentages = crime_matrix_percentages.fillna(0)

print("Proportional crime profiles:")
crime_matrix_percentages.head(3)

Proportional crime profiles:


Crime type,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
MSOA code,,,,,,,,,,,,,,
E02000001,0.039633,0.025144,0.029192,0.025996,0.019604,0.002557,0.307905,0.001492,0.034733,0.035159,0.106115,0.231408,0.023652,0.117409
E02000002,0.246918,0.002538,0.022843,0.049674,0.087382,0.010515,0.059101,0.005439,0.052574,0.016316,0.010515,0.008339,0.108774,0.319072
E02000003,0.205150,0.004707,0.047065,0.068106,0.031285,0.011074,0.073367,0.003876,0.057586,0.037652,0.034053,0.017165,0.113511,0.295404


<br>

Apply **Standard Scaler** (this ensures all crime types contribute equally to distance calculations)

In [14]:
# Standardise the data (important for K-means)
scaler = StandardScaler()
crime_matrix_scaled = scaler.fit_transform(crime_matrix_percentages)

# Convert back to DataFrame for easier handling (since fit_transform returns a NumPy array)
crime_matrix_scaled_df = pd.DataFrame(
    crime_matrix_scaled,
    index=crime_matrix_percentages.index,
    columns=crime_matrix_percentages.columns
)

print(f"Scaled feature matrix shape: {crime_matrix_scaled_df.shape}")
print(f"Mean of scaled features (should be ~0): {crime_matrix_scaled_df.mean().mean():.4f}")
print(f"Std of scaled features (should be ~1): {crime_matrix_scaled_df.std().mean():.4f}")

print("\nScaled crime profiles:")
crime_matrix_scaled_df.head(3)

Scaled feature matrix shape: (1002, 14)
Mean of scaled features (should be ~0): 0.0000
Std of scaled features (should be ~1): 1.0005

Scaled crime profiles:


Crime type,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
MSOA code,,,,,,,,,,,,,,
E02000001,-3.632102,1.151576,-1.315957,-1.930876,-1.108070,-0.972943,7.040934,-1.471408,-1.570150,0.948195,1.008432,4.251980,-1.777317,-2.380416
E02000002,0.598932,-0.878499,-1.661411,-0.407855,3.417239,-0.007473,-0.846996,0.568878,0.166845,-0.804684,-0.783266,-0.666857,0.043207,1.231694
E02000003,-0.253636,-0.683764,-0.343595,0.777798,-0.328179,0.060387,-0.394725,-0.238913,0.654742,1.180161,-0.342123,-0.472245,0.144498,0.807769


<br>
<br>
<br>

### Step 3: Perform K-Means clustering

K-means clustering will group our 1000+ London neighbourhoods into distinct types based on their crime patterns. Think of it like sorting neighbourhoods into categories: "these areas all have similar crime profiles."

Let's start by looking for 5 different neighbourhood types (we'll explore how to choose this number in the bonus section):

In [15]:
# Perform K-means clustering
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

# Fit the model and get cluster assignments
clusters = kmeans.fit_predict(crime_matrix_scaled_df)

# Add cluster labels back to our data
crime_pivot_with_clusters = crime_matrix.copy()
crime_pivot_with_clusters['Cluster'] = clusters
crime_matrix_percentages['Cluster'] = clusters

# Quick summary of what we found
print(f"Found {n_clusters} neighborhood types")
print(f"\nNumber of areas in each cluster:")
cluster_counts = pd.Series(clusters).value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    print(f"  Cluster {cluster_id}: {count} areas")

Found 5 neighborhood types

Number of areas in each cluster:
  Cluster 0: 166 areas
  Cluster 1: 201 areas
  Cluster 2: 260 areas
  Cluster 3: 70 areas
  Cluster 4: 305 areas


<br>
<br>
<br>

### Step 4: Understanding the clusters

Now let's discover what makes each neighbourhood type unique. We'll look at:
1. Their typical crime composition
2. What distinguishes them from the London average
3. How we might name these clusters for policy discussions

What makes each cluster unique? Let's look at their average crime profiles:

In [16]:
# Calculate average crime profile for each cluster
cluster_profiles = crime_matrix_percentages.groupby('Cluster').mean()

# Also get the overall London average for comparison
london_average = crime_matrix_percentages.drop('Cluster', axis=1).mean()

# Find the defining characteristics of each cluster
print("CLUSTER CHARACTERISTICS")
print("="*50)

for cluster_id in range(n_clusters):
    print(f"\n🏷️ Cluster {cluster_id} ({cluster_counts[cluster_id]} areas)")
    print("-"*30)
    
    # Get this cluster's profile
    profile = cluster_profiles.loc[cluster_id]
    
    # Find top 3 crime types
    top_crimes = profile.nlargest(3)
    print("Most common crimes:")
    for crime, pct in top_crimes.items():
        print(f"  • {crime}: {pct:.1%}")
    
    # Find what makes this cluster unique (biggest difference from average)
    avg_profile = crime_matrix_percentages.drop('Cluster', axis=1).mean()
    difference = profile - avg_profile
    most_distinctive = difference.abs().nlargest(1).index[0]
    
    if difference[most_distinctive] > 0:
        print(f"  Distinguished by HIGH {most_distinctive}")
    else:
        print(f"  Distinguished by LOW {most_distinctive}")

CLUSTER CHARACTERISTICS

🏷️ Cluster 0 (166 areas)
------------------------------
Most common crimes:
  • Violence and sexual offences: 21.7%
  • Anti-social behaviour: 19.7%
  • Shoplifting: 13.8%
  Distinguished by HIGH Shoplifting

🏷️ Cluster 1 (201 areas)
------------------------------
Most common crimes:
  • Violence and sexual offences: 23.1%
  • Anti-social behaviour: 22.0%
  • Vehicle crime: 15.9%
  Distinguished by HIGH Vehicle crime

🏷️ Cluster 2 (260 areas)
------------------------------
Most common crimes:
  • Anti-social behaviour: 25.5%
  • Violence and sexual offences: 24.3%
  • Vehicle crime: 8.3%
  Distinguished by HIGH Anti-social behaviour

🏷️ Cluster 3 (70 areas)
------------------------------
Most common crimes:
  • Anti-social behaviour: 16.8%
  • Theft from the person: 16.6%
  • Violence and sexual offences: 16.2%
  Distinguished by HIGH Theft from the person

🏷️ Cluster 4 (305 areas)
------------------------------
Most common crimes:
  • Violence and sexual offen

<br>

<br>
<br>

### Step 5: Comparing clusters visually

#### Chart 1: Heatmap

TIP: Try some other [colour schemes](https://vega.github.io/vega/docs/schemes/) (or define your own)

In [19]:
# Create a heatmap visualisation of cluster profiles
cluster_profile_long = cluster_profiles.reset_index().melt(
    id_vars='Cluster',
    var_name='Crime Type',
    value_name='Percentage'
)

heatmap = alt.Chart(cluster_profile_long).mark_rect(stroke='white', strokeWidth=0.2).encode(
    x=alt.X('Cluster:O').title('Neighbourhood cluster').axis(labelAngle=0, labelFontSize=13),
    y=alt.Y('Crime Type:N').title(None),
    color=alt.Color('Percentage:Q').scale(scheme='magma', reverse=True).title('% of total crime').legend(format='.0%'),
    tooltip=['Cluster', 'Crime Type', alt.Tooltip('Percentage:Q', format='.1%')]
).properties(
    width=300,
    height=350,
    title='Crime patterns by neighbourhood type'
)

heatmap

alt.Chart(...)

<br>
<br>

#### Chart 2: Relative differences

Let's create a side-by-side comparison of how each cluster differs from the London average:

In [40]:
# Calculate differences from London average
differences = cluster_profiles.subtract(london_average, axis=1)

# Prepare data for visualisation
diff_data = []
for cluster_id in range(n_clusters):
    for crime_type in differences.columns:
        diff_data.append({
            'Cluster': f"Cluster {cluster_id}",
            'Crime Type': crime_type,
            'Difference': differences.loc[cluster_id, crime_type]
        })

diff_df = pd.DataFrame(diff_data)

# Create diverging bar chart
diverging_chart = alt.Chart(diff_df).mark_bar().encode(
    x=alt.X('Difference:Q').title('vs London average').scale(domain=[-0.1, 0.1]).axis(format='%'),
    y=alt.Y('Crime Type:N').title(None),
    color=alt.condition(
        alt.datum.Difference > 0,
        alt.value('#ff6b6b'),  # Red for above average
        alt.value('#4dabf7')   # Blue for below average
    ),
    facet=alt.Facet('Cluster:N', columns=2, title='').header(labelFontSize=12, labelPadding=5),
    tooltip=[
        alt.Tooltip('Crime Type:N'),
        alt.Tooltip('Difference:Q', format='.1%', title='Difference (pp)')
    ]
).properties(
    width=220,
    height=180,
    title=alt.TitleParams('How each cluster differs from London average', fontSize=14, offset=12, anchor='middle')
)

diverging_chart.configure_facet(
    spacing=10
)

alt.Chart(...)

Cluster 3 shows clear prevalence of theft across most types, while cluster 0 has significantly higher shoplifting rates without raises theft rates in other areas.

<br>
<br>
<br>

#### Chart 3: **Mapping**

Let's visualise our clusters on a London MSOA map.

In [38]:
# Define data object with Topojson: London MSOA map on map-data repository
map_data = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/map-data/refs/heads/main/gbr/cities/london/MSOA_2021.json', 'geog')

alt.Chart(map_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.2
).encode(
    color=alt.Color('Cluster:O').scale(
        # scheme='set3'
        range=["#4d9c63da", '#FECEAB', '#69aab5', '#2A363B', "#E84A5FCA"]
    ).legend(labelFontSize=13, symbolSize=300)
).properties(
    width=650,
    height=400,
    title='Crime patterns by neighbourhood type'
).transform_lookup(
    lookup='properties.id',
    from_=alt.LookupData(crime_matrix_percentages.reset_index(), 'MSOA code', ['Cluster'])
)

alt.Chart(...)

<br>

### Policy insights

Based on our clustering analysis, what could be some takeaways for London policing?

1. **Resource allocation**: Different neighbourhood types need different policing strategies
2. **Peer learning**: Neighbourhoods in the same cluster can share effective interventions
3. **Performance metrics**: Compare neighbourhoods within their cluster for fairer assessment

Any other ideas?

In [196]:
# Identify high-risk clusters
violence_by_cluster = cluster_profiles['Violence and sexual offences']
high_violence_cluster = violence_by_cluster.idxmax()
print(f"\nHighest violence rates: Cluster {high_violence_cluster}")
print(f"  Priority for community safety initiatives")


Highest violence rates: Cluster 4
  Priority for community safety initiatives


If you want to take this further, you might want to introduce more variables, such as bringing in socio-economic data.

<br>
<br>
<br>
<br>


---


<br>
<br>

### BONUS: Determine optimal number of clusters - **Elbow Method** & **Silhouette Score**

The **Elbow Method**: We plot the within-cluster sum of squares (WCSS) for different values of K. The "elbow" point where the curve bends suggests a good balance between model fit and complexity.

The **Silhouette Score** measures how distinct our clusters are from each other. 

<br>

> Go to the next notebook (kmeans2) to see how we can use these methods.